In [1]:
""" 
Capital Asset Pricing Model and Security Markey Line
R_i = R_f = B_i(E[R_mkt] - R_f), where R_i marks the returns, beta is B_i (which measures the systematic risk of stock)
This equation measures the relationship between risk and stock returns for every stock in the portfolio basket.
"""
from scipy import stats
stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = \
    stats.linregress(stock_returns, mkt_returns)
print(alpha, beta)

-0.008481900352462384 0.5077431878770808


In [3]:
""" 
Arbitrary Pricing Theory Model uses linear regression (least squares regression) to address the market risk factor
E[R_i] = a_i + B_ijF_j wiht 1<=j<=infinity
"""
import numpy as np
import statsmodels.api as sm

# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8) \
                       for i in range(num_periods)])

y_values = all_values[:, 0]
x_values = all_values[:, 1:]
x_values = sm.add_constant(x_values)
results = sm.OLS(y_values, x_values).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.690
Model:                            OLS   Adj. R-squared:                 -1.477
Method:                 Least Squares   F-statistic:                    0.3184
Date:                Wed, 06 Jul 2022   Prob (F-statistic):              0.880
Time:                        15:38:52   Log-Likelihood:                 3.3055
No. Observations:                   9   AIC:                             9.389
Df Residuals:                       1   BIC:                             10.97
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1405      2.640      0.432      0.7

C:\Users\ayanj\Projects\FinanceWithPython\my_env\lib\site-packages\scipy\stats\_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [5]:
""" 
Linear optimization has to do with rising securities in portfolios, so limitations are brought up
Linear optimization solves this, by focusing on variables, with the PuLP library assisting in this
"""
import pulp

x = pulp.LpVariable('x', lowBound=0)
y = pulp.LpVariable('y', lowBound=0)

problem = pulp.LpProblem(
    'A simple maximization objective', 
    pulp.LpMaximize)
problem += 3*x + 2*y, 'The objective function'
problem += 2*x + y <= 100, '1st constraint'
problem += x + y <= 80, '2nd constraint'
problem += x <= 40, '3rd constraint'
problem.solve()

print("Maximization Results")
for variable in problem.variables():
    print(variable.name, '=', variable.varValue)

C:\Users\ayanj\Projects\FinanceWithPython\my_env\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Maximization Results
x = 20.0
y = 60.0


In [6]:
"""
Linear optimization with integer programming abd binary conditions
"""
import pulp

dealers = ['X', 'Y', 'Z']
variable_costs = {'X': 500, 'Y': 350, 'Z': 450}
fixed_costs = {'X': 4000, 'Y': 2000, 'Z': 6000}

# Define PuLP variables to solve
quantities = pulp.LpVariable.dicts('quantity', 
                                   dealers, 
                                   lowBound=0,
                                   cat=pulp.LpInteger)
is_orders = pulp.LpVariable.dicts('orders', 
                                  dealers,
                                  cat=pulp.LpBinary)

model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)
model += sum(
    [variable_costs[i]*quantities[i] + \
         fixed_costs[i]*is_orders[i] for i in dealers])\
    , 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150\
    ,  'Total contracts required'
model += is_orders['X']*30 <= quantities['X'] <= \
    is_orders['X']*100, 'Boundary of total volume of X'
model += is_orders['Y']*30 <= quantities['Y'] <= \
    is_orders['Y']*90, 'Boundary of total volume of Y'
model += is_orders['Z']*30 <= quantities['Z'] <= \
    is_orders['Z']*70, 'Boundary of total volume of Z'
model.solve()

print('Minimization Results:')
for variable in model.variables():
    print(variable, '=', variable.varValue)

print('Total cost:',  pulp.value(model.objective))

Minimization Results:
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
Total cost: 66500.0


In [7]:
"""
The LU Decomposition solves square systems of linear equations
"""
import numpy as np
import scipy.linalg as linalg
import scipy


# Define A and B
A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0.]])
B = np.array([4., 5., 6.])

LU = linalg.lu_factor(A)
x = linalg.lu_solve(LU, B)

print('Solving for x between A and B matrices, ', x)

P, L, U = scipy.linalg.lu(A)
print('Solving LU for matrix A')
print('P=\n', P)
print('L=\n', L)
print('U=\n', U)

Solving for x between A and B matrices,  [  6.  15. -23.]
Solving LU for matrix A
P=
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
L=
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.5 -0.2  1. ]]
U=
 [[ 2.   1.   1. ]
 [ 0.   2.5  1.5]
 [ 0.   0.  -0.2]]


In [9]:
"""
The Cholesky decomposition is another way of solving systems of linear equations,
and this can be faster than LU by exploiting symmetric matrices
Matrix A must be decomposed as A = LL^T, where L is a lower triangular matrix
with real and positive numbers, while L^T is a conjugate transpose of L
"""
import numpy as np

A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0., 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])

L = np.linalg.cholesky(A)

print(L)


[[ 3.16227766  0.          0.          0.        ]
 [-0.31622777  3.3015148   0.          0.        ]
 [ 0.63245553 -0.24231301  3.08889696  0.        ]
 [ 0.          0.9086738  -0.25245792  2.6665665 ]]


In [12]:
"""
If the size of one of the matrices is large, it is easier to iteratiely
break down the problem so that it is easier to converge. The first method is 
the Jacobian method
"""
import numpy as np

def jacobi (A, B, n, tol = 1e-10):
    x = np.zeros_like(B)

    for iter_count in range(n):
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]
        if np.allclose(x, x_new, tol):
            break
        
        x = x_new
    return x
A = np.array([
    [10., -1., 2., 0.], 
    [-1., 11., -1., 3.], 
    [2., -1., 10., -1.], 
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 25
x = jacobi(A, B, n)
print('x', '=', x)

x = [ 1.  2. -1.  1.]


In [13]:
"""
If the size of one of the matrices is large, it is easier to iteratiely
break down the problem so that it is easier to converge. The first method is 
the Gausee-Seidel method
"""
import numpy as np

def gauss(A, B, n, tol = 1e-10):
    L = np.tril(A)
    U = A-L
    L_inv = np.linalg.inv(L)
    x = np.zeros_like(B)
    
    for i in range(n):
        Ux = np.dot(U, x)
        x_new = np.dot(L_inv, B - Ux)
        
        if np.allclose(x, x_new, tol):
            break
        x = x_new
        
    return x

A = np.array([
    [10., -1., 2., 0.], 
    [-1., 11., -1., 3.], 
    [2., -1., 10., -1.], 
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 100
x = gauss(A, B, n)
print('x', '=', x)

x = [ 1.  2. -1.  1.]
